In [1]:
ip = get_ipython()
ibe = ip.configurables[-1]
ibe.figure_formats = { 'pdf', 'png'}

In [5]:
%matplotlib inline

from dot_pattern_decoder.dot_pattern_decoder import DotPatternDecoder
from dot_pattern_decoder.printer import Printer
from dot_pattern_decoder.constants import Constants

decoder = DotPatternDecoder(Path.cwd().joinpath('../2020-04-08_22-00-09.jpg'))

decoder.get_position_codes(decoder.preprocess_image())
decoder.correction_position_2(decoder.x_codes, decoder.y_codes)
decoder.correction_position_2(decoder.x_codes, decoder.x_codes)

ModuleNotFoundError: No module named 'constants'

# Декодирование позиции

## Обработка изображения


В умных ручках используются инфракрасные камеры и прозрачные в этом спектре
чернила. За счет этого чернила не мешают позиционироваться на листах при письме.
Камера же делает снимки только тогда, когда на стержень оказывается давление.
Поэтому задача оцифровки рукописного текста и изображений в случае умных ручек
сводится к правильному декодированию последовательности таких снимков в координаты.


Первым этапом декодирования, после получения изображения, содержащего блок
минимального размера (8x8), является препроцессинг. **Исходное изображение (а)** 
переводится в **оттенки серого (б)**, а потом **бинаризуется (в)** (значение порога выведено 
эмпирически). Из получившегося изображения необходимо **удалить шумы (г)**, чтобы убрать 
артефакты печати и повысить качество распознавания. Для этого используется медианный 
фильтр (значение которого также выведено эмпирически).

In [ ]:
Printer.plot_images([
    (decoder.raw_image, 'Исходное (а)'), 
    (decoder.image_grayscale, 'Оттенки серого (б)'),
    (decoder.black_white_image, f'ЧБ с порогом {threshold_for_bw} (в)'),
])

In [ ]:
deleted_noise_image = black_white_image.filter(ImageFilter.MedianFilter(3))
pixels_array = np.array(deleted_noise_image.convert(mode='1'))
centroids = get_centroids(pixels_array) 

centroids_with_neighbours = []
for point in centroids:
    x, y = point
    offsets = [-1, 0, 1]
    for offset_x in offsets:
        for offset_y in offsets:
            centroids_with_neighbours.append((x + offset_x, y + offset_y))

centroids_zero_array = np.zeros_like(deleted_noise_image)
for y in range(deleted_noise_image.height):
    for x in range(deleted_noise_image.width):
        centroids_zero_array[y, x] = [255, 255]

centroids_image = points_to_image(centroids_zero_array, centroids_with_neighbours)

Printer.plot_images([
    (deleted_noise_image, 'После удаления шума (г)'),
    (points_to_image(deleted_noise_image, centroids_with_neighbours), 'ЧБ с центроидами (а)'),
    (centroids_image, 'Только центроиды (б)')
])

Нужно отметить, что в данном изложении алгоритма используются относительно выровненные изрбражения.
Ручка же делает снимки под наклоном в силу того, что это происходит при письме. Причем важно учесть,
что угол наклона может варьироваться очень сильно, что делает позиционирование невозможным без
выравнивания изображения.

Точки исходного изображения представляются на бинаризованном изображении в виде областей белых пикселей.
С помощью инструмента regionprops из пакета skimage находим центроиды данных областей (**изображение (а)** показывает бинарное изображение с наложенными центроидами, а **изображение (б)** только центроиды). 

Важно отметить, что в случае неправильной фильтрации изображения, слишком низком пороге для бинаризации, некачественного исходного снимка, либо поврежденного бумажного носителя может возникнуть ситуация, когда возникнут лишние точки, либо пропадут нужные. В случае первых двух проблем нужно провести эксперименты для вычисления оптимальных параметров, а в случае других - попробовать поискать на изображении другой валидный блок, если таких нет, а коррекция ошибок не справляется - спозиционироваться невозможно.

In [ ]:
import cv2

def filter_with_threshold(matrix, threshold):
    filtered = np.zeros_like(matrix)
    points = []
    for y in range(len(filtered)):
        raw = []
        need_append = False
        for x in range(len(filtered[y])):
            if matrix[y, x] > threshold:
                filtered[y, x] = matrix[y, x]
                raw.append((x, y))
                need_append = True
                
        if need_append:
            points.append(raw)
                
                
    return (np.log(1+np.abs(filtered)), points)


def fft(image_for_fft, threshold):
    image_for_cv = cv2.cvtColor(np.array(image_for_fft), cv2.COLOR_RGB2GRAY)    

    img_c2 = np.fft.fft2(image_for_cv)
    img_c3 = np.fft.ifftshift(img_c2)
    img_c5, points = filter_with_threshold(img_c3, img_c3.max() * threshold)
#   img_c5, points = img_c3, points
    
    return img_c5, points


image_fft, fft_points = fft(raw_image.convert(mode='RGB'), 0.06)
ret, image_fft = cv2.threshold(image_fft, 1, 255, cv2.THRESH_BINARY)

Printer.plot_images([
    ( raw_image, "Original Image" ),
    ( image_fft, "After FFT with threshold" )
])

In [ ]:
N = int(math.sqrt(len(centroids)))
centroids_in_grid = centroids[np.argsort(centroids[:, 0])].reshape(N, N, 2)

for i in range(N):
    centroids_in_grid[i] = centroids_in_grid[i][np.argsort(centroids_in_grid[i][:, 1])]
    
x_coordinates_of_centroids = centroids_in_grid[:, :, 0]
average_x = x_coordinates_of_centroids.mean(axis = 1)

In [ ]:
y_coordinates_of_centroids = centroids_in_grid[:, :, 1]
average_y = y_coordinates_of_centroids.mean(axis = 0)

In [ ]:
from PIL import ImageDraw

def add_virtual_lines(image, values_x, values_y):
    virtual_lines = image.convert(mode='RGB')
    drawer = ImageDraw.Draw(virtual_lines)
    color = (0, 255, 255)
    for x in values_x:
        drawer.line([(x, 0), (x, virtual_lines.height)], fill=color, width=1)

    for y in values_y:
        drawer.line([(0, y), (virtual_lines.width, y)], fill=color, width=1)
    
    return virtual_lines

Printer.plot_images([
    (add_virtual_lines(deleted_noise_image, average_y, average_x), ''),
    (add_virtual_lines(centroids_image, average_y, average_x), '')
])

## Вычисление позиций точек

Координаты точек раскладываются в две таблицы - одна для координат X (Таблица 1), 
а вторая для координат Y (Таблица 2). 

In [ ]:
print('Таблица 1. X-координаты точек')
Printer.print_table(x_coordinates_of_centroids)

In [ ]:
print('Таблица 2. Y-координаты точек')
Printer.print_table(y_coordinates_of_centroids)

В **Таблице 1** не просто так отмечены столбцы, а в **Таблице 2** - строки. Это связано с тем, как кодируется информация о положении в данном шаблоне. На что конкретно это влияет будет показано ниже, сейчас же достаточно знать, что позиция на листе закодирована таким образом: информация о столбце в координатах X, а о строке в координатах Y. 

In [ ]:
print('Таблица 3. Некоторые параметры X-диапозонов')

max_x = x_coordinates_of_centroids.max(axis = 1)
min_x = x_coordinates_of_centroids.min(axis = 1)
difference_x = max_x - min_x

Printer.print_x_params(min_x, max_x, difference_x, average_x)

In [ ]:
print('Таблица 4. Некоторые параметры Y-диапозонов')

max_y = y_coordinates_of_centroids.max(axis = 0)
min_y = y_coordinates_of_centroids.min(axis = 0)
difference_y = max_y - min_y

Printer.print_y_params(min_y, max_y, difference_y, average_y)

### Позиции точек и коррекция ошибок распознавания


У точек есть 4 варианта смещения относительно номинальной позиции (пересечение вертикальной и горизонтальной линии). 

![img](img/image002.jpg)

Следовательно, каждая точка кодирует 2 бита информации. Один из битов для кодирования столбца (далее называется x бит), а второй - для строки (y бит).

In [ ]:
Printer.print_named_rows(constants.BITS_TABLE, ['x bit', 'y bit'])

Из относительных положений внутри одного столбца точки можно разделить на 3 позиции: левая (L), средняя (M) и правая (R), в зависимости от того, в каком направлении смещена точка относительно оси X, либо оси Y. Обозначим эти позиици через { -1, 0, 1 } соответственно. По таблицу приведенной ниже видно, что позиции вверх/вниз и влево/вправо неразличимы при проекции на ось X и ось Y соответственно. 

In [ ]:
Printer.print_named_rows(constants.DIRECTIONS_TABLE, ['x bit', 'y bit'])

В основном, в диапозонах есть точки всех типо, но в редких случаях там присутвуют точки только двух видов (L и M, либо M и R). Такой случай необходимо обрабатывать отдельно. Таким образом, все столбцы можно раздить на классы A и B. Для точек из класса А присваиваются коды, как было сказано выше. А для точек из класа B все так же, но расстояние между максимумом и минимумом делится на два интервала, а коды меняются { -2, 2 }. 

In [ ]:
threshold_for_class = 2/3 * difference_x.max()
classes_x = [ 'A' if diff > threshold_for_class else 'B' for diff in difference_x ]
# print(threshold_for_class)

threshold_for_class = 2/3 * difference_y.max()
classes_y = [ 'A' if diff > threshold_for_class else 'B' for diff in difference_y ]
# print(threshold_for_class)


print('Таблица 5. Классы X-диапозонов')
Printer.print_named_rows({'class': classes_x}, [f'x_{i}' for i in range(N)])
print()

print('Таблица 6. Классы Y-диапозонов')
Printer.print_named_rows({'class': classes_y}, [f'y_{i}' for i in range(N)])

In [ ]:
def split_into_three_intervals(minimum, maximum):
    diff = maximum - minimum
    border_1 = minimum + 1/3 * diff
    border_2 = minimum + 2/3 * diff
    return ((minimum, border_1), (border_1, border_2), (border_2, maximum))


def split_into_two_intervals(minimum, maximum):
    diff = maximum - minimum
    border_1 = minimum + 1/2 * diff
    return ((minimum, border_1),(border_1, maximum))


def value_belongs_to_interval(value, interval):
    return interval[0] <= value and value <= interval[1]


def get_position_code(value, intervals, codes):
    for i in range(len(intervals)):
        if value_belongs_to_interval(value, intervals[i]):
            return codes[i]
            

def split_into_intervals(minimum, maximum, klass):
    if klass == 'A':
        return split_into_three_intervals(minimum, maximum)
    else:
        return split_into_two_intervals(minimum, maximum)

    
def get_position_codes(coordinates, minimums, maximums, klasses):    
    intervals = [
        split_into_intervals(minimum, maximum, klass)
        for minimum, maximum, klass in zip(minimums, maximums, klasses)
    ]
#     display(tabulate.tabulate(intervals, [], tablefmt="html"))

    codes = np.zeros_like(coordinates)
    for i in range(len(codes)):
        for j in range(len(codes[i])):
            value = coordinates[i,j]
            interval = intervals[i]
            posibles_codes = [-1, 0, 1] if klasses[i] == 'A' else [-2, 2]
            codes[i,j] = get_position_code(value, interval, posibles_codes)
    
    return codes


x_codes = get_position_codes(x_coordinates_of_centroids, min_x, max_x, classes_x)

In [ ]:
y_codes = get_position_codes(y_coordinates_of_centroids.transpose(), min_y, max_y, classes_y).transpose()
for i in range(len(y_codes)):
    for j in range(len(y_codes)):
        if y_codes[i, j] != 0: 
            y_codes[i, j] = -1 * y_codes[i, j]

Printer.print_directions(x_codes, y_codes)

Для коррекции таблицы воспользуемся таблицей, приведенной ниже.

In [ ]:
Printer.print_vertical_table([ 
        [ '-1/1' ,  '0'   ,     '-'      ,  '-'            ],
        [ '-1/1' , '-2/2' , 'sign(in_1)' ,  '0'            ],
        [ '0'    , '-2/2' ,    '0'       ,  'sign(in_2)'   ],
        [ '1/-1' , '-1/1' ,    'D'       ,  'D'            ],
        [ '0'    , '0'    ,    'D'       ,  'D'            ],
        [ '2/-2' , '-2/2' ,     '-'      ,  '-'            ]
    ], ['in 1', 'in 2', 'out 1', 'out 2']
)

In [ ]:
def replace(source, replacements):
    destination = np.zeros_like(source)
    for i in range(len(destination)):
        for what, to in replacements:
            if source[i] == what:
                destination[i] = to 
                break
            else: 
                destination[i] = source[i]
       
    return destination


def replace_code_in_line(line, code, to):
    replace(line, [(code, to), (-1 * code, sign(-1 * y_code) * to)])

    
def correction_position_2(x_lines, y_lines):
    for j in range(len(x_lines)):
        x_line, y_line = x_lines[j], y_lines[j]
        for i, (x_code, y_code) in enumerate(zip(x_line, y_line)):
            if (abs(y_code) == 2):
                if abs(x_code) == 1:
                    y_lines[:, i] = replace(
                        y_lines[:, i], [(y_code, 0), (-1 * y_code, -1 * np.sign(y_code))]
                    )
                elif abs(x_code) == 0:
                    y_lines[:, i] = replace(
                        y_lines[:, i], [(y_code, np.sign(y_code)), (-1 * y_code, 0)]
                    )
                
                
x_codes_corrected, y_codes_corrected = x_codes.copy(), y_codes.copy() 

correction_position_2(x_codes_corrected, y_codes_corrected)
correction_position_2(y_codes_corrected, x_codes_corrected)

Printer.print_named_rows(constants.DIRECTIONS_TABLE, ['x bit', 'y bit'])

Printer.print_directions(x_codes_corrected.transpose(), y_codes_corrected.transpose())

Printer.plot_images([
    (add_virtual_lines(deleted_noise_image, average_y, average_x), ''),
])

In [ ]:
def filter_by_class(classes, widths, target_class):
    return list(map(
            lambda v: v[1],
            filter(lambda c: c[0] == target_class, zip(classes, widths))
        )
    )


def get_avg_by_class(classes, target_class, widths):
    return np.array(list(filter_by_class(classes, widths, target_class))).mean()


def get_new_position(coordinate, width, avg_width):
    return coordinate - width / 2


def correction_virtual_lines(lines_coordinate, classes, widths):
    avg_for_class_a = get_avg_by_class(classes, 'A', widths)
    new_coordinates = [ 
        get_new_position(coordinate, width, avg_for_class_a) 
        for coordinate, width
        in zip(
            filter_by_class(classes, lines_coordinate, 'B'), 
            filter_by_class(classes, widths, 'B')
        ) 
    ]
    j = 0
    for i in range(len(lines_coordinate)):
        if classes[i] == 'B':
            lines_coordinate[i] = new_coordinates[j]
            j += 1
            
    return lines_coordinate

lines_after_correction = correction_virtual_lines(average_x.copy(), classes_x, difference_x)

print(average_x)
print(classes_x)
print(lines_after_correction)

Printer.plot_images([
    (add_virtual_lines(centroids_image, average_x, []), ''),
    (add_virtual_lines(centroids_image, lines_after_correction, []), ''),
])

In [ ]:

# print(line_x_coordinate)
reconstructed_x_lines = reconstruct_line(x_codes_corrected, x_coordinates_of_centroids)
reconstructed_y_lines = reconstruct_line(
    y_codes_corrected.transpose(), 
    y_coordinates_of_centroids.transpose()
)


display(tabulate.tabulate(
    [['X'] + reconstructed_x_lines], [f'line {i}' for i in range(N)], tablefmt="html"
))

display(tabulate.tabulate(
    [['Y'] + reconstructed_y_lines], [f'line {i}' for i in range(N)], tablefmt="html"
))
plot_images([
    (add_virtual_lines(deleted_noise_image, reconstructed_x_lines, reconstructed_x_lines), ''),
    (add_virtual_lines(centroids_image, reconstructed_x_lines, reconstructed_x_lines), '')
])